## Introduction
In the previous post, we read about the concepts of __Graph__ and __Session__ which describes the way the data flows in TensorFlow. In this tutorial, we'll take a look at some of the __Tensor Types__ used in TensorFlow and specially the ones commonly used in creating neural network models, namely ___Constant___, ___Variable___, and ___Placeholder___. 

This will also enable us to shed light on some of the points and questions left unanswered in the previous post.

Remember that we need to import the TensorFlow library at the very beginning of our code using:

In [2]:
import tensorflow as tf




## 1. Constant 

As the name speaks for itself, __Constants__ are used as constant value tensors. They create a node that takes a constant value. You can simply create a constant tensor using __tf.constant__. It accepts the following arguments:


In [ ]:
tf.constant(value, dtype=None, shape=None, name='Const', verify_shape=False)

Now let's look at a very simple example.

### Example 1:
Let's create two constants and add them together. Constant tensors can be defined simply by defining a value:

In [ ]:
# create graph
a = tf.constant(2)
b = tf.constant(3)
c = a + b
# launch the graph in a session
with tf.Session() as sess:
    print(sess.run(c))

Perfect! Now Let's look at the created graph and generated data types:
<img src="files/files/2_1.png" width="1000" height="2000" >
___Fig1. ___ __Left:__ generated graph visualized in Tensorboard, __Right:__ generated variables (screenshot captured from PyCharm debugger when running in debug mode)

As it's depicted in the figure, we created 3 tensors with __"Python-names"__ _a_, _b_, and _c_. However, we didn't define any __"TensorFlow-name"__ for them. Therefore, TensorFlow assigns some default names to them which are depicted in the graph: __const__ and __const_1__ for the input constants and __add__ for the output of the addition operation. We can easily modify it and define our own names, like:

In [ ]:
# create graph
a = tf.constant(2, name='A')
b = tf.constant(3, name='B')
c = tf.add(a, b, name='Sum')
# launch the graph in a session
with tf.Session() as sess:
    print(sess.run(c))

This time the graph and created tensors are as follows:
<img src="files/files/2_2.png" width="1000" height="2000" >
___Fig2. ___ generated graph (Left) and variables (Right) with the modified names


We can also define constants of different types (integer, float, etc.) and shapes (vectors, matrices, etc.).


### Example 2:

In [ ]:
s = tf.constant(2.3, name='scalar', dtype=tf.float32)
m = tf.constant([[1, 2], [3, 4]], name='matrix')
# launch the graph in a session
with tf.Session() as sess:
    print(sess.run(s))
    print(sess.run(m))

## 2. VARIABLE
Variables are stateful nodes which output their current value; meaning that they can retain their value over multiple executions of a graph. They have a number of useful features such as:

- They can be __saved__ to your disk during and after training. This allows people from different companies and groups to save, restore and send over their model parameters to other people.
- By default, gradient updates (used in all neural networks) will apply to all variables in your graph. In fact, variables are the things that you want to tune in order to minimize the loss. 

These features make variables suitable to be used as the network parameters (i.e. weights and biases).

You might ask what are the differences between variables and constants? Well there are two major differences:

1. Constants are (guess what), constants. Their value doesn't change. You'd usually need your network parameters to be updated and that's where the __variable__ comes into play.

2. Constants are stored in the graph definition which makes them memory-expensive. In other words, constants with millions of entries makes the graph loading much slower.


Again, it's important to remember that creating a variables is an operation (look at the Fig. 2 of the first tutorial for a quick recap). When we evaluate these operations in the session, we'll get the output value of the operations.

### 2.1. Create Variables
To create a variable, you can use __tf.Variable__ as:


In [ ]:
# Create a variable.
w = tf.Variable(<initial-value>, name=<optional-name>)

Some examples of creating scalar and matrix variables are as follows:

In [ ]:
s = tf.Variable(2, name="scalar") 
m = tf.Variable([[1, 2], [3, 4]], name="matrix") 
W = tf.Variable(tf.zeros([784,10]))

Variable __W__ defined above will create a matrix with 784 rows and 10 columns which will be initialized by zeros. This can be used as a weight matrix of a feed-forward neural network (or even in a linear regression model) from a layer with 784 neuron to a layer with 10 neuron. We'll see more of this later in this turorial.

__*Note:__ we use tf.Variable() with uppercase "V", and tf.constant with lowercase "c". You don't necessarily need to know the reason, but it's simply because tf.constant is an op, while tf.Variable is a class with multiple ops.

__* IMPORTANT Note:__ Calling tf.Variable to create a variable is the old way of creating a variable. TensorFlow recommends to use the wraper __tf.get_variable__ which accepts the name, shape, etc as its arguments as follows:


In [ ]:
tf.get_variable(name,
                shape=None,
                dtype=None,
                initializer=None,
                regularizer=None,
                trainable=True,
                collections=None,
                caching_device=None,
                partitioner=None,
                validate_shape=True,
                use_resource=None,
                custom_getter=None,
                constraint=None)

Some examples are as follows:

In [ ]:
s = tf.get_variable("scalar", initializer=tf.constant(2)) 
m = tf.get_variable("matrix", initializer=tf.constant([[0, 1], [2, 3]]))
W = tf.get_variable("weight_matrix", shape=(784, 10), initializer=tf.zeros_initializer())

### 2.2. Initialize Variables
Variables need to be initialized befor being used. To do so, we have to invoke a __variable initializer operation__ and run the operation on the session. This is the easiest way to initialize variables which initializes all variables at once.

The following toy example shows how we can add an op to initialize the variables.

### Example 3:
create two variables and add them together. Then print out their value and the summation result.

In [ ]:
a = tf.get_variable(name="var_1", initializer=tf.constant(2))
b = tf.get_variable(name="var_2", initializer=tf.constant(3))
c = tf.add(a, b, name="Add1")

# launch the graph in a session
with tf.Session() as sess:
    # now let's evaluate their value
    print(sess.run(a))
    print(sess.run(b))
    print(sess.run(c))

As you can see, we ran into __FailedPreconditionError: Attempting to use uninitialized value__. This is because we tried to evaluate the variables before initializing them. Let's correct the code by first initializing all variables and then evaluating them.

In [3]:
# create graph
a = tf.get_variable(name="A", initializer=tf.constant(2))
b = tf.get_variable(name="B", initializer=tf.constant(3))
c = tf.add(a, b, name="Add")
# add an Op to initialize global variables
init_op = tf.global_variables_initializer()

# launch the graph in a session
with tf.Session() as sess:
    # run the variable initializer operation
    sess.run(init_op)
    # now let's evaluate their value
    print(sess.run(a))
    print(sess.run(b))
    print(sess.run(c))

2
3
5


Let's take a quick look at the graph and generated variables:
<img src="files/files/2_3.png" width="1000" height="2000" >
___Fig3. ___ generated graph (Left) and variables (Right)

As you can see, two blue boxes are the generated variables (compare them with constant nodes in Fig. 2) which are added together using "Add" operation.


__*Note:__ Variables are usually used for weights and biases in neural networks.

- __weights__ are usually initialized from a normal distribution using `tf.truncated_normal_initializer()`.

- __biases__ are usually initialized from zeros using `tf.zeros_initializer()`.

Let's look at a very simple example of creating weight and bias variables with proper initialization:

### Example 4:
Create the weight and bias matrices for a fully-connected layer with 2 neuron to another layer with 3 neuron.
In this scenario, the weight and bias variables must be of size $[2, 3]$ and 3 respectively.

In [5]:
# create graph
weights = tf.get_variable(name="W", shape=[2,3], initializer=tf.truncated_normal_initializer(stddev=0.01))
biases = tf.get_variable(name="b", shape=[3], initializer=tf.zeros_initializer())

# add an Op to initialize global variables
init_op = tf.global_variables_initializer()

# launch the graph in a session
with tf.Session() as sess:
    # run the variable initializer
    sess.run(init_op)
    # now we can run our operations
    W, b = sess.run([weights, biases])
    print('weights = ', W)
    print('biases = ', b)

ValueError: Variable W already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-4-531aeccfa637>", line 2, in <module>
    weights = tf.get_variable(name="W", shape=[2,3], initializer=tf.truncated_normal_initializer(stddev=0.01))
  File "/home/cougarnet.uh.edu/amobiny/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/home/cougarnet.uh.edu/amobiny/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2822, in run_ast_nodes
    if self.run_code(code, result):


## PLACEHOLDER:
Placeholders are tensors that are placed to hold the data.
We can build our graph without needing the data (because data is huge).
In the time of need, we can feed the data in the right place (guess what place ?!!)

BUT, Placeholders are just holding the place... where should we feed the input?

In a dictionaty called __feed_dict__.





In [ ]:
# create graph
# create a placeholder of type float 32-bit, value is a vector of 3 elements
a = tf.placeholder(tf.float32, shape=[3])
# create a constant of type float 32-bit, value is a vector of 3 elements
b = tf.constant([5, 5, 5], tf.float32)
c = a+b

# launch the graph in a session
with tf.Session() as sess:
    # create a feed_dict:
    feed_dict={a: [1, 2, 3]}
    # feed it to placeholder a via the dict 
    print(sess.run(c, feed_dict=feed_dict)) 

## Exercise:
Now let's see a cool example. We will load an image and slice a part of the image and visualize it.

We will load an image and try to slice it using the _slice_ method in tensorflow. Complete the code to run the session.
Complete the code and 


In [ ]:

# load the image
filename = 'flowers.jpg'
raw_image_data = mpimg.imread(filename)

# create a placeholder for the image
image = tf.placeholder(dtype="uint8", shape=[None, None, 3])
# slice the image 
slice = tf.slice(image, begin=[1000, 0, 0], size=[200, 200, -1])

# launch the graph in a session
result = []
with tf.Session() as session:
    ######################## YOUR CODE HERE ##########################
    # Hint: You should run the session and pass the op that you want # 
    # and the feed_dict and store the value in "result" variable.    #
    #                                                                #
    pass
    #                                                                #
    ##################################################################
    
plt.imshow(raw_image_data)
plt.title('Original image')
plt.show()

plt.imshow(result)
plt.title('Cropped image')
plt.show()